In [1]:
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm
import dotenv

os.chdir(dotenv.find_dotenv().replace('.env', ''))
from src.data_correction import pseudo_batch_transform, shift, pseudo_batch_transform_pandas
from pseudobatch.datasets import load_standard_fedbatch

In [2]:
dat = load_standard_fedbatch()
print(dat.columns)
print(dat.shape)

Index(['Kc_s', 'mu_max', 'Yxs', 'Yxp', 'Yxco2', 'F0', 'mu0', 's_f', 'Ki_p',
       'sample_volume', 'p', 'timestamp', 'm_Glucose', 'm_Biomass',
       'm_Product', 'm_CO2', 'v_Volume', 'v_Feed_accum', 'c_Glucose',
       'c_Biomass', 'c_Product', 'c_CO2', 'c_Volume', 'c_Feed', 'mu_true'],
      dtype='object')
(1011, 25)


In [3]:
# calculate consumed glucose
initial_m_glucose = dat['m_Glucose'].iloc[0]
glucose_conc_in_feed = eval(dat.p[0])[7]
dat['m_consumed_Glucose'] = initial_m_glucose + dat['v_Feed_accum'] * glucose_conc_in_feed - dat['m_Glucose']  
concentration_in_feed = [0, glucose_conc_in_feed, 0]

species_to_correct = ["c_Biomass", "c_Glucose", "c_Product"]

dat[['c_Biomass_pseudo', 'c_Glucose_pseudo', 'c_Product_pseudo']]= pseudo_batch_transform_pandas(
    dat,
    ['c_Biomass', 'c_Glucose', 'c_Product'],
    'v_Volume',
    'v_Feed_accum',
    concentration_in_feed,
    'sample_volume'
)
dat_samples = dat.query("sample_volume > 0")


Produce input data for the excels sheet

In [4]:
dat_samples[['timestamp', 'c_Biomass', 'c_Glucose', 'c_Product', 'v_Feed_accum', 'v_Volume', 'sample_volume']]

,timestamp,c_Biomass,c_Glucose,c_Product,v_Feed_accum,v_Volume,sample_volume
167,10.000000,1.315046,0.117208,0.676000,15.906036,1015.906036,100.0
244,14.545455,2.026109,0.171224,1.265827,28.960902,928.960902,100.0
320,19.090909,3.083525,0.296168,2.143140,47.314262,847.314262,100.0
397,23.636364,4.568350,0.651151,3.375809,72.816654,772.816654,100.0
474,28.181818,6.410392,1.735493,4.907704,107.795685,707.795685,100.0
550,32.727273,8.322732,4.439107,6.504606,155.117789,655.117789,100.0
627,37.272727,9.979404,9.564975,7.899563,218.291500,618.291500,100.0
704,41.818182,11.131987,17.411957,8.887827,301.721794,601.721794,100.0
780,46.363636,11.644800,27.577468,9.355220,411.318522,611.318522,100.0
857,50.909091,11.528932,38.997586,9.306527,555.738787,655.738787,100.0


In [10]:
print(glucose_conc_in_feed)

100.0
